In [1]:
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np


from tensorflow.keras.layers import LSTM

H:\Users\Gwydion\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
H:\Users\Gwydion\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
H:\Users\Gwydion\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
H:\Users\Gwydion\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub

In [2]:
df_spectra=pd.read_csv("spectrum_energy_intensity2.csv",index_col=[0])
df_spectra.head(-50)

,Energy,Intensity,k6a1,k6a2,k11,k12,k9a1,k9a2
0,0.874,0.019935,0.25,0.25,0.25,0.25,0.25,0.25
1,0.213,0.060488,0.25,0.25,0.25,0.25,0.25,0.50
2,0.866,0.020153,0.25,0.25,0.25,0.25,0.25,0.75
3,0.867,0.019641,0.25,0.25,0.25,0.25,0.50,0.25
4,0.220,0.060471,0.25,0.25,0.25,0.25,0.50,0.50
...,...,...,...,...,...,...,...,...
674,0.868,0.021991,0.75,0.75,0.25,0.75,0.75,0.75
675,0.865,0.021101,0.75,0.75,0.50,0.25,0.25,0.25
676,0.213,0.060723,0.75,0.75,0.50,0.25,0.25,0.50
677,0.868,0.019078,0.75,0.75,0.50,0.25,0.25,0.75


In [3]:
numpy_input=df_spectra[["k6a1","k6a2","k11","k12","k9a1","k9a2"]].to_numpy()
numpy_energy_intensity=df_spectra[["Energy","Intensity"]].to_numpy()


df_complete=pd.DataFrame(numpy_energy_intensity)
df_training=df_spectra.sample(frac=0.8,random_state=20)
df_testing=df_spectra.drop(df_training.index)

In [4]:
feature_columns_A = []

k6a1 = tf.feature_column.numeric_column("k6a1")
feature_columns_A.append(k6a1)

k6a2 = tf.feature_column.numeric_column("k6a2")
feature_columns_A.append(k6a2)

k11 = tf.feature_column.numeric_column("k11")
feature_columns_A.append(k11)

k12 = tf.feature_column.numeric_column("k12")
feature_columns_A.append(k12)

k9a1 = tf.feature_column.numeric_column("k9a1")
feature_columns_A.append(k9a1)

k9a2 = tf.feature_column.numeric_column("k9a2")
feature_columns_A.append(k9a2)

my_feature_layer_A = tf.keras.layers.DenseFeatures(feature_columns_A)


In [5]:
#define the plotting function.

def plot_the_loss_curve(epochs, mse,mse_test=0):
    """Plot a curve of loss vs. epoch."""

    plt.figure()
    plt.xlabel("Epoch")
    plt.ylabel("Mean Squared Error")

    plt.plot(epochs, mse, label="Loss")
    if(mse_test!=0):
        plt.plot(epochs,mse_test)
    plt.legend()
    plt.ylim([mse.min()*0.95, mse.max() * 1.03])
    plt.show()  

print("Defined the plot_the_loss_curve function.")

Defined the plot_the_loss_curve function.


In [29]:
def create_model(my_learning_rate, my_feature_layer,unit_layer_list=[10,12],regulations=0.04):
    """Create and compile a regression model."""
    model = tf.keras.models.Sequential()
    # Add the layer containing the feature columns to the model.
    model.add(my_feature_layer)

    # Describe the topography of the model by calling the tf.keras.layers.Dense
    # method once for each layer. We've specified the following arguments:
    #   * units specifies the number of nodes in this layer.
    #   * activation specifies the activation function (Rectified Linear Unit).
    #   * name is just a string that can be useful when debugging.

    # Define the first hidden layer with 10 nodes.   
    for n in unit_layer_list:
        print(n)
        model.add(tf.keras.layers.Dense(units=n, activation='relu', 
                                  kernel_regularizer=tf.keras.regularizers.l2(regulations),name='Hidden'+str(n)))
                
   
   

    # Define the output layer.
    model.add(tf.keras.layers.Dense(units=2,  
                                    name='Output'))                              
  
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=my_learning_rate),
                loss="mean_squared_error",
                metrics=[tf.keras.metrics.MeanSquaredError()])
    return model



In [30]:
def train_model(model, dataset, epochs, label_name,
                batch_size=None):
    """Train the model by feeding it data."""

    # Split the dataset into features and label.
    features = {name:np.array(value) for name, value in dataset.items()}
    label = [np.array(features.pop(label)) for label in label_name]
    history = model.fit(x=features, y=label, batch_size=batch_size,
                      epochs=epochs, shuffle=True) 

    # The list of epochs is stored separately from the rest of history.
    epochs = history.epoch
  
    # To track the progression of training, gather a snapshot
    # of the model's mean squared error at each epoch. 
    hist = pd.DataFrame(history.history)
    mse = hist["mean_squared_error"]

    return epochs, mse

In [56]:
# The following variables are the hyperparameters.
learning_rate = 0.0001
epochs_ini = 8000
batch_size = 500

# Specify the label
label_name = ["Energy","Intensity"]

#try different things:

my_model = create_model(learning_rate, my_feature_layer_A,unit_layer_list=[10,15,10],regulations=1e-7)

epochs, mse = train_model(my_model, df_training, epochs_ini, 
                          label_name, batch_size)



10
15
10
Epoch 1/8000
2/2 [==============================] - 0s 1ms/step - loss: 0.3974 - mean_squared_error: 0.3974
Epoch 2/8000
2/2 [==============================] - 0s 500us/step - loss: 0.3965 - mean_squared_error: 0.3965
Epoch 3/8000
2/2 [==============================] - 0s 500us/step - loss: 0.3957 - mean_squared_error: 0.3957
Epoch 4/8000
2/2 [==============================] - 0s 500us/step - loss: 0.3948 - mean_squared_error: 0.3948
Epoch 5/8000
2/2 [==============================] - 0s 499us/step - loss: 0.3940 - mean_squared_error: 0.3940
Epoch 6/8000
2/2 [==============================] - 0s 1000us/step - loss: 0.3931 - mean_squared_error: 0.3931
Epoch 7/8000
2/2 [==============================] - 0s 1000us/step - loss: 0.3923 - mean_squared_error: 0.3923
Epoch 8/8000
2/2 [==============================] - 0s 500us/step - loss: 0.3914 - mean_squared_error: 0.3914
Epoch 9/8000
2/2 [==============================] - 0s 500us/step - loss: 0.3906 - mean_squared_error: 0.3906
E

2/2 [==============================] - 0s 1ms/step - loss: 0.3453 - mean_squared_error: 0.3453
Epoch 76/8000
2/2 [==============================] - 0s 1000us/step - loss: 0.3446 - mean_squared_error: 0.3446
Epoch 77/8000
2/2 [==============================] - 0s 501us/step - loss: 0.3440 - mean_squared_error: 0.3440
Epoch 78/8000
2/2 [==============================] - 0s 1ms/step - loss: 0.3434 - mean_squared_error: 0.3434
Epoch 79/8000
2/2 [==============================] - 0s 501us/step - loss: 0.3427 - mean_squared_error: 0.3427
Epoch 80/8000
2/2 [==============================] - 0s 500us/step - loss: 0.3421 - mean_squared_error: 0.3421
Epoch 81/8000
2/2 [==============================] - 0s 1ms/step - loss: 0.3415 - mean_squared_error: 0.3414
Epoch 82/8000
2/2 [==============================] - 0s 500us/step - loss: 0.3408 - mean_squared_error: 0.3408
Epoch 83/8000
2/2 [==============================] - 0s 500us/step - loss: 0.3402 - mean_squared_error: 0.3401
Epoch 84/8000
2/2 [=

2/2 [==============================] - 0s 500us/step - loss: 0.2893 - mean_squared_error: 0.2893
Epoch 149/8000
2/2 [==============================] - 0s 1ms/step - loss: 0.2884 - mean_squared_error: 0.2884
Epoch 150/8000
2/2 [==============================] - 0s 500us/step - loss: 0.2875 - mean_squared_error: 0.2875
Epoch 151/8000
2/2 [==============================] - 0s 1ms/step - loss: 0.2867 - mean_squared_error: 0.2867
Epoch 152/8000
2/2 [==============================] - 0s 500us/step - loss: 0.2858 - mean_squared_error: 0.2858
Epoch 153/8000
2/2 [==============================] - 0s 500us/step - loss: 0.2849 - mean_squared_error: 0.2849
Epoch 154/8000
2/2 [==============================] - 0s 1000us/step - loss: 0.2841 - mean_squared_error: 0.2841
Epoch 155/8000
2/2 [==============================] - 0s 500us/step - loss: 0.2832 - mean_squared_error: 0.2832
Epoch 156/8000
2/2 [==============================] - 0s 500us/step - loss: 0.2824 - mean_squared_error: 0.2824
Epoch 157/

Epoch 221/8000
2/2 [==============================] - 0s 500us/step - loss: 0.2246 - mean_squared_error: 0.2245
Epoch 222/8000
2/2 [==============================] - 0s 1ms/step - loss: 0.2237 - mean_squared_error: 0.2237
Epoch 223/8000
2/2 [==============================] - 0s 500us/step - loss: 0.2228 - mean_squared_error: 0.2228
Epoch 224/8000
2/2 [==============================] - 0s 500us/step - loss: 0.2219 - mean_squared_error: 0.2219
Epoch 225/8000
2/2 [==============================] - 0s 1ms/step - loss: 0.2211 - mean_squared_error: 0.2211
Epoch 226/8000
2/2 [==============================] - 0s 1000us/step - loss: 0.2202 - mean_squared_error: 0.2202
Epoch 227/8000
2/2 [==============================] - 0s 1ms/step - loss: 0.2193 - mean_squared_error: 0.2193
Epoch 228/8000
2/2 [==============================] - 0s 501us/step - loss: 0.2184 - mean_squared_error: 0.2184
Epoch 229/8000
2/2 [==============================] - 0s 999us/step - loss: 0.2175 - mean_squared_error: 0.21

2/2 [==============================] - 0s 501us/step - loss: 0.1678 - mean_squared_error: 0.1678
Epoch 295/8000
2/2 [==============================] - 0s 1ms/step - loss: 0.1672 - mean_squared_error: 0.1672
Epoch 296/8000
2/2 [==============================] - 0s 1000us/step - loss: 0.1666 - mean_squared_error: 0.1666
Epoch 297/8000
2/2 [==============================] - 0s 500us/step - loss: 0.1660 - mean_squared_error: 0.1660
Epoch 298/8000
2/2 [==============================] - 0s 1000us/step - loss: 0.1654 - mean_squared_error: 0.1654
Epoch 299/8000
2/2 [==============================] - 0s 1ms/step - loss: 0.1647 - mean_squared_error: 0.1647
Epoch 300/8000
2/2 [==============================] - 0s 1000us/step - loss: 0.1641 - mean_squared_error: 0.1641
Epoch 301/8000
2/2 [==============================] - 0s 1ms/step - loss: 0.1635 - mean_squared_error: 0.1635
Epoch 302/8000
2/2 [==============================] - 0s 500us/step - loss: 0.1629 - mean_squared_error: 0.1629
Epoch 303/

...

2/2 [==============================] - 0s 1ms/step - loss: 0.0595 - mean_squared_error: 0.0595
Epoch 7655/8000
2/2 [==============================] - 0s 500us/step - loss: 0.0595 - mean_squared_error: 0.0595
Epoch 7656/8000
2/2 [==============================] - 0s 1ms/step - loss: 0.0595 - mean_squared_error: 0.0595
Epoch 7657/8000
2/2 [==============================] - 0s 1ms/step - loss: 0.0595 - mean_squared_error: 0.0595
Epoch 7658/8000
2/2 [==============================] - 0s 500us/step - loss: 0.0595 - mean_squared_error: 0.0595
Epoch 7659/8000
2/2 [==============================] - 0s 500us/step - loss: 0.0595 - mean_squared_error: 0.0595
Epoch 7660/8000
2/2 [==============================] - 0s 999us/step - loss: 0.0595 - mean_squared_error: 0.0595
Epoch 7661/8000
2/2 [==============================] - 0s 1ms/step - loss: 0.0595 - mean_squared_error: 0.0595
Epoch 7662/8000
2/2 [==============================] - 0s 500us/step - loss: 0.0595 - mean_squared_error: 0.0595
Epoch 7

Epoch 7727/8000
2/2 [==============================] - 0s 500us/step - loss: 0.0595 - mean_squared_error: 0.0595
Epoch 7728/8000
2/2 [==============================] - 0s 1ms/step - loss: 0.0595 - mean_squared_error: 0.0595
Epoch 7729/8000
2/2 [==============================] - 0s 500us/step - loss: 0.0595 - mean_squared_error: 0.0595
Epoch 7730/8000
2/2 [==============================] - 0s 500us/step - loss: 0.0595 - mean_squared_error: 0.0595
Epoch 7731/8000
2/2 [==============================] - 0s 1000us/step - loss: 0.0595 - mean_squared_error: 0.0595
Epoch 7732/8000
2/2 [==============================] - 0s 500us/step - loss: 0.0595 - mean_squared_error: 0.0595
Epoch 7733/8000
2/2 [==============================] - 0s 1ms/step - loss: 0.0595 - mean_squared_error: 0.0595
Epoch 7734/8000
2/2 [==============================] - 0s 500us/step - loss: 0.0595 - mean_squared_error: 0.0595
Epoch 7735/8000
2/2 [==============================] - 0s 1ms/step - loss: 0.0595 - mean_squared_er

2/2 [==============================] - 0s 1000us/step - loss: 0.0594 - mean_squared_error: 0.0594
Epoch 7800/8000
2/2 [==============================] - 0s 500us/step - loss: 0.0594 - mean_squared_error: 0.0594
Epoch 7801/8000
2/2 [==============================] - 0s 500us/step - loss: 0.0594 - mean_squared_error: 0.0594
Epoch 7802/8000
2/2 [==============================] - 0s 1ms/step - loss: 0.0594 - mean_squared_error: 0.0594
Epoch 7803/8000
2/2 [==============================] - 0s 500us/step - loss: 0.0594 - mean_squared_error: 0.0594
Epoch 7804/8000
2/2 [==============================] - 0s 1ms/step - loss: 0.0594 - mean_squared_error: 0.0594
Epoch 7805/8000
2/2 [==============================] - 0s 500us/step - loss: 0.0594 - mean_squared_error: 0.0594
Epoch 7806/8000
2/2 [==============================] - 0s 500us/step - loss: 0.0594 - mean_squared_error: 0.0594
Epoch 7807/8000
2/2 [==============================] - 0s 501us/step - loss: 0.0594 - mean_squared_error: 0.0594
Ep

2/2 [==============================] - 0s 1000us/step - loss: 0.0595 - mean_squared_error: 0.0595
Epoch 7872/8000
2/2 [==============================] - 0s 1ms/step - loss: 0.0594 - mean_squared_error: 0.0594
Epoch 7873/8000
2/2 [==============================] - 0s 500us/step - loss: 0.0594 - mean_squared_error: 0.0594
Epoch 7874/8000
2/2 [==============================] - 0s 500us/step - loss: 0.0594 - mean_squared_error: 0.0594
Epoch 7875/8000
2/2 [==============================] - 0s 501us/step - loss: 0.0594 - mean_squared_error: 0.0594
Epoch 7876/8000
2/2 [==============================] - 0s 1ms/step - loss: 0.0594 - mean_squared_error: 0.0594
Epoch 7877/8000
2/2 [==============================] - 0s 1ms/step - loss: 0.0594 - mean_squared_error: 0.0594
Epoch 7878/8000
2/2 [==============================] - 0s 1ms/step - loss: 0.0594 - mean_squared_error: 0.0594
Epoch 7879/8000
2/2 [==============================] - 0s 1ms/step - loss: 0.0594 - mean_squared_error: 0.0594
Epoch 78

2/2 [==============================] - 0s 500us/step - loss: 0.0594 - mean_squared_error: 0.0594
Epoch 7945/8000
2/2 [==============================] - 0s 1000us/step - loss: 0.0594 - mean_squared_error: 0.0594
Epoch 7946/8000
2/2 [==============================] - 0s 500us/step - loss: 0.0594 - mean_squared_error: 0.0594
Epoch 7947/8000
2/2 [==============================] - 0s 1000us/step - loss: 0.0594 - mean_squared_error: 0.0594
Epoch 7948/8000
2/2 [==============================] - 0s 500us/step - loss: 0.0594 - mean_squared_error: 0.0594
Epoch 7949/8000
2/2 [==============================] - 0s 999us/step - loss: 0.0594 - mean_squared_error: 0.0594
Epoch 7950/8000
2/2 [==============================] - 0s 500us/step - loss: 0.0594 - mean_squared_error: 0.0594
Epoch 7951/8000
2/2 [==============================] - 0s 500us/step - loss: 0.0594 - mean_squared_error: 0.0594
Epoch 7952/8000
2/2 [==============================] - 0s 1000us/step - loss: 0.0594 - mean_squared_error: 0.0

In [57]:
test_features = {name:np.array(value) for name, value in df_testing.items()}
test_label = [np.array(test_features.pop(label)) for label in label_name]


#print(test_label)
example_mydata = df_testing.sample(frac=0.8,random_state=20)
#run the evaluation
# i checked and they should be correct
example_features = {name:np.array(value) for name, value in example_mydata.items()}
#print(example_features)

predicted = 2*my_model.predict(example_features)-1

#print(predicted)


predicted=predicted.reshape(len(predicted),2)
#print(predicted)
predicted_1=[predicted[i][0] for i in range(len(predicted))]
predicted_2=[predicted[i][1] for i in range(len(predicted))]


print(predicted[:])
print(predicted_1[:5])
print(predicted_2[:5])

exact_energy = example_mydata["Energy"].to_numpy()*2-1
exact_intensity = example_mydata["Intensity"].to_numpy()



print("Intensity:",my_model.evaluate(x = test_features, y = test_label, batch_size=batch_size))

df_compare=pd.DataFrame({"predicted1":predicted_1,"predicted2":predicted_2, "Reference Energy":exact_energy, "Reference Intensity":exact_intensity})

#df_compare.plot.scatter(x="exact", y="predicted")
df_compare.to_csv("first_test.csv")
df_compare.head(50)

[[ 0.12792337  0.13733697]
 [-0.5834267  -0.5921892 ]
 [-0.6181369  -0.61607087]
 [-0.37051862 -0.37098283]
 [ 0.5154853   0.5010493 ]
 [-0.47900665 -0.4854859 ]
 [-0.42198247 -0.4214905 ]
 [-0.53103745 -0.5145483 ]
 [-0.26627815 -0.27506244]
 [-0.5800526  -0.56834716]
 [ 0.05916893  0.07042253]
 [ 0.11720622  0.11030281]
 [-0.51592445 -0.5115983 ]
 [ 0.48908138  0.49237227]
 [-0.52238065 -0.5180285 ]
 [ 0.53796184  0.5342369 ]
 [ 0.5644742   0.5650822 ]
 [ 0.25714445  0.26043034]
 [ 0.5727571   0.56440115]
 [ 0.23474121  0.24091077]
 [-0.56707954 -0.56839496]
 [ 0.00492036  0.01922381]
 [-0.50355923 -0.52868855]
 [ 0.20926261  0.20342803]
 [ 0.01579714  0.0100168 ]
 [ 0.33578742  0.33016086]
 [ 0.01595259  0.03863978]
 [ 0.00256431  0.01813614]
 [ 0.23327708  0.25143027]
 [ 0.35447693  0.335042  ]
 [ 0.1127032   0.12589407]
 [-0.33854032 -0.35121578]
 [ 0.70633495  0.6915879 ]
 [ 0.56228745  0.5605538 ]
 [-0.01593715 -0.00184393]
 [ 0.29895353  0.30745816]
 [-0.5311196  -0.5442127 ]
 

,predicted1,predicted2,Reference Energy,Reference Intensity
0,0.127923,0.137337,0.732,0.021675
1,-0.583427,-0.592189,-0.632,0.074461
2,-0.618137,-0.616071,-0.630,0.076080
3,-0.370519,-0.370983,-0.756,0.001645
4,0.515485,0.501049,0.176,0.063414
5,-0.479007,-0.485486,-0.630,0.076080
6,-0.421982,-0.421490,-0.630,0.076000
7,-0.531037,-0.514548,-0.632,0.074461
8,-0.266278,-0.275062,-0.756,0.001604
9,-0.580053,-0.568347,-0.630,0.076485
